In [ ]:
/content/Dataset/iam-handwriting-word-database.zip

NameError: name 'content' is not defined

InferenceModel

In [ ]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Set your Kaggle API credentials (you need to have a Kaggle account)
os.environ["KAGGLE_USERNAME"] = "chirrashyamprakash"
os.environ["KAGGLE_KEY"] = "3ba910b04abfb4b38532c945fc8fa45a"

# Instantiate the Kaggle API
api = KaggleApi()
# Verify that the Kaggle API is authenticated
api.authenticate()

# Download the dataset
dataset_name = "nibinv23/iam-handwriting-word-database"
output_path = "/content/Dataset"  # Specify the path where you want to save the dataset

# Download the dataset
api.dataset_download_files(dataset_name, path=output_path, unzip=True)


OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.

In [ ]:
import tarfile
from tqdm import tqdm
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
import os

def download_and_unzip(url, extract_to='Datasets', chunk_size=1024*1024):
    # Download the file
    http_response = urlopen(url)

    # Read the content in chunks
    data = b''
    iterations = http_response.length // chunk_size + 1
    for _ in tqdm(range(iterations)):
        data += http_response.read(chunk_size)

    # Extract the zip file
    with ZipFile(BytesIO(data)) as zipfile:
        zipfile.extractall(path=extract_to)

# Define the dataset path
dataset_path = '/content/Dataset/IAM_Words'

# Check if dataset exists, if not download and unzip
if not os.path.exists(dataset_path):
    # Download and unzip the dataset
    download_and_unzip('https://git.io/J0fjL', extract_to='/content/Dataset')
    # Extract words.tgz
    with tarfile.open(os.path.join(dataset_path, "words.tgz")) as file:
        file.extractall(path=dataset_path)


100%|██████████| 784/784 [05:40<00:00,  2.30it/s]


In [ ]:
dataset_path = '/content/Dataset/IAM_Words'

# Preprocess the dataset
words_file_path = os.path.join(dataset_path, "words.txt")
with open(words_file_path, "r") as words_file:
    lines = words_file.readlines()

for line in tqdm(lines):
    if line.startswith("#"):
        continue

    line_split = line.split(" ")
    if line_split[1] == "err":
        continue

    folder1 = line_split[0][:3]
    folder2 = line_split[0][:8]
    file_name = line_split[0] + ".png"
    label = line_split[-1].rstrip('\n')

    rel_path = os.path.join(dataset_path, "words", folder1, folder2, file_name)
    if not os.path.exists(rel_path):
        continue

    dataset.append([rel_path, label])
    vocab.update(list(label))
    max_len = max(max_len, len(label))

# Now you can continue with the rest of your code using `dataset`, `vocab`, and `max_len`.


100%|██████████| 115338/115338 [00:01<00:00, 106746.92it/s]


In [ ]:
import os
from datetime import datetime
import os
import yaml


class BaseModelConfigs:
    def __init__(self):
        self.model_path = None

    def serialize(self):
        class_attributes = {key: value
                            for (key, value)
                            in type(self).__dict__.items()
                            if key not in ['__module__', '__init__', '__doc__', '__annotations__']}
        instance_attributes = self.__dict__

        # first init with class attributes then apply instance attributes overwriting any existing duplicate attributes
        all_attributes = class_attributes.copy()
        all_attributes.update(instance_attributes)

        return all_attributes

    def save(self, name: str = "configs.yaml"):
        if self.model_path is None:
            raise Exception("Model path is not specified")

        # create directory if not exist
        if not os.path.exists(self.model_path):
            os.makedirs(self.model_path)

        with open(os.path.join(self.model_path, name), "w") as f:
            yaml.dump(self.serialize(), f)

    @staticmethod
    def load(configs_path: str):
        with open(configs_path, "r") as f:
            configs = yaml.load(f, Loader=yaml.FullLoader)

        config = BaseModelConfigs()
        for key, value in configs.items():
            setattr(config, key, value)

        return config

class ModelConfigs(BaseModelConfigs):
    def __init__(self):
        super().__init__()
        self.model_path = os.path.join("/content/extracted/202301111911")
        self.vocab = ""
        self.height = 32
        self.width = 128
        self.max_text_length = 0
        self.batch_size = 16
        self.learning_rate = 0.0005
        self.train_epochs = 1000
        self.train_workers = 20

In [ ]:
# Create a ModelConfigs object to store model configurations
configs = ModelConfigs()

# Save vocab and maximum text length to configs
configs.vocab = "".join(vocab)
configs.max_text_length = max_len
configs.save()

In [ ]:
import zipfile

# Path to the zip file
zip_file_path = "/content/202301111911-20240515T134220Z-001.zip"

# Directory where you want to extract the files
extracted_dir = "/content/extracted/"

# Create the directory if it doesn't exist
import os
os.makedirs(extracted_dir, exist_ok=True)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

print("Zip file has been extracted to:", extracted_dir)

Zip file has been extracted to: /content/extracted/


In [ ]:
import os
import time
import typing
import numpy as np
import onnxruntime as ort
from collections import deque

class FpsWrapper:
    """ Decorator to calculate the frames per second of a function
    """
    def __init__(self, func: typing.Callable):
        self.func = func
        self.fps_list = deque([], maxlen=100)

    def __call__(self, *args, **kwargs):
        start = time.time()
        results = self.func(self.instance, *args, **kwargs)
        self.fps_list.append(1 / (time.time() - start))
        self.instance.fps = np.mean(self.fps_list)
        return results

    def __get__(self, instance, owner):
        self.instance = instance
        return self.__call__.__get__(instance, owner)


class OnnxInferenceModel:
    """ Base class for all inference models that use onnxruntime

    Attributes:
        model_path (str, optional): Path to the model folder. Defaults to "".
        force_cpu (bool, optional): Force the model to run on CPU or GPU. Defaults to GPU.
        default_model_name (str, optional): Default model name. Defaults to "model.onnx".
    """
    def __init__(
        self,
        model_path: str = "",
        force_cpu: bool = False,
        default_model_name: str = "model.onnx",
        *args, **kwargs
        ):
        self.model_path = model_path.replace("\\", "/")
        self.force_cpu = force_cpu
        self.default_model_name = default_model_name

        # check if model path is a directory with os path
        if os.path.isdir(self.model_path):
            self.model_path = os.path.join(self.model_path, self.default_model_name)

        if not os.path.exists(self.model_path):
            raise Exception(f"Model path ({self.model_path}) does not exist")

        providers = ["CUDAExecutionProvider", "CPUExecutionProvider"] if ort.get_device() == "GPU" and not force_cpu else ["CPUExecutionProvider"]

        self.model = ort.InferenceSession(self.model_path, providers=providers)

        self.metadata = {}
        if self.model.get_modelmeta().custom_metadata_map:
            # add metadata to self object
            for key, value in self.model.get_modelmeta().custom_metadata_map.items():
                try:
                    new_value = eval(value) # in case the value is a list or dict
                except:
                    new_value = value
                self.metadata[key] = new_value

        # Update providers priority to only CPUExecutionProvider
        if self.force_cpu:
            self.model.set_providers(["CPUExecutionProvider"])

        self.input_shapes = [meta.shape for meta in self.model.get_inputs()]
        self.input_names = [meta.name for meta in self.model._inputs_meta]
        self.output_names = [meta.name for meta in self.model._outputs_meta]

    def predict(self, data: np.ndarray, *args, **kwargs):
        raise NotImplementedError

    @FpsWrapper
    def __call__(self, data: np.ndarray):
        results = self.predict(data)
        return results

In [ ]:
import typing
import numpy as np
from itertools import groupby


def ctc_decoder(predictions: np.ndarray, chars: typing.Union[str, list]) -> typing.List[str]:
    """ CTC greedy decoder for predictions

    Args:
        predictions (np.ndarray): predictions from model
        chars (typing.Union[str, list]): list of characters

    Returns:
        typing.List[str]: list of words
    """
    # use argmax to find the index of the highest probability
    argmax_preds = np.argmax(predictions, axis=-1)

    # use groupby to find continuous same indexes
    grouped_preds = [[k for k,_ in groupby(preds)] for preds in argmax_preds]

    # convert indexes to chars
    texts = ["".join([chars[k] for k in group if k < len(chars)]) for group in grouped_preds]

    return texts


def edit_distance(prediction_tokens: typing.List[str], reference_tokens: typing.List[str]) -> int:
    """ Standard dynamic programming algorithm to compute the Levenshtein Edit Distance Algorithm

    Args:
        prediction_tokens: A tokenized predicted sentence
        reference_tokens: A tokenized reference sentence
    Returns:
        Edit distance between the predicted sentence and the reference sentence
    """
    # Initialize a matrix to store the edit distances
    dp = [[0] * (len(reference_tokens) + 1) for _ in range(len(prediction_tokens) + 1)]

    # Fill the first row and column with the number of insertions needed
    for i in range(len(prediction_tokens) + 1):
        dp[i][0] = i

    for j in range(len(reference_tokens) + 1):
        dp[0][j] = j

    # Iterate through the prediction and reference tokens
    for i, p_tok in enumerate(prediction_tokens):
        for j, r_tok in enumerate(reference_tokens):
            # If the tokens are the same, the edit distance is the same as the previous entry
            if p_tok == r_tok:
                dp[i+1][j+1] = dp[i][j]
            # If the tokens are different, the edit distance is the minimum of the previous entries plus 1
            else:
                dp[i+1][j+1] = min(dp[i][j+1], dp[i+1][j], dp[i][j]) + 1

    # Return the final entry in the matrix as the edit distance
    return dp[-1][-1]

def get_cer(
    preds: typing.Union[str, typing.List[str]],
    target: typing.Union[str, typing.List[str]],
    ) -> float:
    """ Update the cer score with the current set of references and predictions.

    Args:
        preds (typing.Union[str, typing.List[str]]): list of predicted sentences
        target (typing.Union[str, typing.List[str]]): list of target words

    Returns:
        Character error rate score
    """
    if isinstance(preds, str):
        preds = [preds]
    if isinstance(target, str):
        target = [target]

    total, errors = 0, 0
    for pred_tokens, tgt_tokens in zip(preds, target):
        errors += edit_distance(list(pred_tokens), list(tgt_tokens))
        total += len(tgt_tokens)

    if total == 0:
        return 0.0

    cer = errors / total

    return cer

def get_wer(
    preds: typing.Union[str, typing.List[str]],
    target: typing.Union[str, typing.List[str]],
    ) -> float:
    """ Update the wer score with the current set of references and predictions.

    Args:
        target (typing.Union[str, typing.List[str]]): string of target sentence or list of target words
        preds (typing.Union[str, typing.List[str]]): string of predicted sentence or list of predicted words

    Returns:
        Word error rate score
    """
    if isinstance(preds, str) and isinstance(target, str):
        preds = [preds]
        target = [target]

    if isinstance(preds, list) and isinstance(target, list):
        errors, total_words = 0, 0
        for _pred, _target in zip(preds, target):
            if isinstance(_pred, str) and isinstance(_target, str):
                errors += edit_distance(_pred.split(), _target.split())
                total_words += len(_target.split())
            else:
                print("Error: preds and target must be either both strings or both lists of strings.")
                return np.inf

    else:
        print("Error: preds and target must be either both strings or both lists of strings.")
        return np.inf

    wer = errors / total_words

    return wer

In [ ]:
# inferenceModel.py
import cv2
import typing
import numpy as np

class ImageToWordModel(OnnxInferenceModel):
    def __init__(self, char_list: typing.Union[str, list], *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.char_list = char_list

    def predict(self, image: np.ndarray):
        image = cv2.resize(image, self.input_shapeS[:2][::-1])

        image_pred = np.expand_dims(image, axis=0).astype(np.float32)

        preds = self.model.run(None, {self.input_name: image_pred})[0]

        text = ctc_decoder(preds, self.char_list)[0]

        return text

if __name__ == "__main__":
    import pandas as pd
    from tqdm import tqdm

    configs = BaseModelConfigs.load("/content/extracted/202301111911/configs.yaml")

    model = ImageToWordModel(model_path=configs.model_path, char_list=configs.vocab)

    df = pd.read_csv("/content/extracted/202301111911/val.csv").values.tolist()

    accum_cer = []
    for image_path, label in tqdm(df):
        image = cv2.imread(image_path)

        prediction_text = model.predict(image)

        cer = get_cer(prediction_text, label)
        print(f"Image: {image_path}, Label: {label}, Prediction: {prediction_text}, CER: {cer}")

        accum_cer.append(cer)

    print(f"Average CER: {np.average(accum_cer)}")

  0%|          | 0/2171 [00:00<?, ?it/s]


AttributeError: 'ImageToWordModel' object has no attribute 'input_shapeS'

In [ ]:
!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 29.8 MB/s eta 0:00:00


In [ ]:
import onnx

# Load the ONNX model
model_path = "/content/extracted/202301111911/model.onnx"
model = onnx.load(model_path)

# Print model summary
print(onnx.helper.printable_graph(model.graph))


graph tf2onnx (
  %input[FLOAT, unk__438x32x128x3]
) initializers (
  %model/output/Tensordot/free:0[INT32, 2]
  %model/output/Tensordot/axes:0[INT32, 1]
  %model/output/Tensordot/ReadVariableOp:0[FLOAT, 256x75]
  %model/output/Tensordot/Const_2:0[INT32, 1]
  %model/output/BiasAdd/ReadVariableOp:0[FLOAT, 75]
  %model/conv2d_9/BiasAdd_weights_fused_bn[FLOAT, 32x32x3x3]
  %model/conv2d_9/BiasAdd_bias_fused_bn[FLOAT, 32]
  %model/conv2d_8/BiasAdd_weights_fused_bn[FLOAT, 32x16x3x3]
  %model/conv2d_8/BiasAdd_bias_fused_bn[FLOAT, 32]
  %model/conv2d_7/BiasAdd_weights_fused_bn[FLOAT, 16x16x3x3]
  %model/conv2d_7/BiasAdd_bias_fused_bn[FLOAT, 16]
  %model/conv2d_6/BiasAdd_weights_fused_bn[FLOAT, 16x16x3x3]
  %model/conv2d_6/BiasAdd_bias_fused_bn[FLOAT, 16]
  %model/conv2d_5/Conv2D/ReadVariableOp:0[FLOAT, 16x16x1x1]
  %model/conv2d_5/BiasAdd/ReadVariableOp:0[FLOAT, 16]
  %model/conv2d_4/BiasAdd_weights_fused_bn[FLOAT, 16x16x3x3]
  %model/conv2d_4/BiasAdd_bias_fused_bn[FLOAT, 16]
  %model/conv2d_

In [ ]:
import onnxruntime
import numpy as np
from PIL import Image
import torchvision.transforms as transforms

# Load the ONNX model
model_path = '/content/extracted/202301111911/model.onnx'
session = onnxruntime.InferenceSession(model_path)

# Preprocess the input image
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize((128, 32)),  # Resize to match model input shape
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = preprocess(image)
    image = image.numpy()
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Make predictions
def predict(image_path):
    input_image = preprocess_image(image_path)
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name
    pred = session.run([output_name], {input_name: input_image})
    return pred

# Example usage
image_path = '/content/Dataset/iam_words/words/a01/a01-000u/a01-000u-00-00.png'
prediction = predict(image_path)
print(prediction)


InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Got invalid dimensions for input: input for the following indices
 index: 1 Got: 3 Expected: 32
 index: 3 Got: 32 Expected: 3
 Please fix either the inputs/outputs or the model.

# Text Identification

In [ ]:
!apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 3s (1,874 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
import cv2
import pytesseract

# Path to your input image in Google Colab
input_image_path = '/content/WhatsApp Image 2024-05-16 at 12.41.25_ebd49ab3.jpg'

# Load the input image using OpenCV
image = cv2.imread(input_image_path)

# Convert the image to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Use Tesseract with the LSTM OCR engine to perform OCR on the image
recognized_text = pytesseract.image_to_string(gray_image, config='--oem 1 --psm 6')

# Print the recognized text
print(recognized_text)


Current Program status Register .
Capy 0 ey ano coe internal Operations.



In [ ]:
!pip install flask transformers

In [ ]:
!pip install openai

In [ ]:
import torch

print(torch.cuda.is_available())

False


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the pre-trained model and tokenizer
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Format the input text as a question
input_text = "Are cats and dogs from the same family?"
question = f"Is it true that {input_text}?"

# Tokenize the input text and convert it to tensors
input_ids = torch.tensor([tokenizer.encode(question, add_special_tokens=True)]).cuda()

# Get the model's prediction
with torch.no_grad():
    outputs = model(input_ids)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1)

# Map the predicted class to a true/false label
label_map = {0: "False", 1: "True"}
label = label_map[predicted_class.item()]

# Print the result
print(f"The statement '{input_text}' is {label}.")

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx